<h1 style="color: rgba(237, 153, 29, 1);">Aula Prática - Implementação de Tarefas</h1>
<h2>Threads</h2>

<small>
<p><strong>IMPORTANTE</strong>: O comando '%%file' é usado no Python para criar arquivos .java no diretório onde este notebook está salvo. Os arquivos criados são nomeados conforme o identificador fornecido após o comando '%%file'.</p>
</small>

<h3>O que é uma Thread?</h3>

<p>Fluxo de execução que opera dentro de um processo ou do núcleo do sistema operacional (também chamada de linha de execução). Por padrão, cada processo contém pelo menos uma thread.</p>

<p>Tipos de threads:</p>

<ul>
    <li><strong style="color: #E0901B;">Threads de usuário</strong>: fluxos de execução dentro de um processo, associados à execução da aplicação.</li>
    <li><strong style="color: #E0901B;">Threads de núcleo</strong>: fluxos de execução dentro do núcleo, representando threads de usuário ou atividades internas do próprio núcleo.</li>
<ul>

<h3>O Modelo 1:1 (Um-para-Um)</h3>

<p>Nesse modelo, cada thread de nível de usuário criada por um programa é mapeada para uma thread a nível de kernel (<strong>Figura 1</strong>). Isso significa que o kernel do sistema operacional está totalmente ciente de cada thread e pode gerenciá-las e agendá-las individualmente.</p>

<p>Este modelo permite que threads de um mesmo processo sejam executadas em paralelo em diferentes núcleos de CPU. Isso maximiza o desempenho em máquinas com múltiplos processadores. Assim, se uma thread faz uma chamada de sistema bloqueante (como ler um arquivo), apenas essa thread é pausada pelo kernel. As outras threads do mesmo processo podem continuar suas execuções.
</p>

<figure>
    <img src="imagem3.png" alt="Modelo de execução de threads 1:1.">
    <figcaption>Figura 1: Modelo de execução de threads 1:1. Baseado em: Maziero (2014).</figcaption>
</figure>

<h4>Desvantagem</h4>

<p>o <strong>modelo 1:1</strong> garante paralelismo real e evita bloqueios globais, mas tem como desvantagem o alto custo de gerenciamento quando há grande número de threads.</p>

<h4 style="color: #2d3436;"><strong>Código 1</strong>: Criando algumas threads.</h4>

In [ ]:
%%file threads.c

// Comando de compilação: gcc -Wall threads.c -o out -lpthread
// Comando de execução: ./out

#include <pthread.h> // Biblioteca para trabalhar com threads POSIX. Essencial para criar, gerenciar e sincronizar threads.
#include <stdio.h>   // Para funções de entrada e saída, como printf().
#include <stdlib.h>  // Para funções de utilidade geral, como exit().
#include <unistd.h>  // Para a função sleep() que pausa a execução.

#define NUM_THREADS 12

// Função que será executada por cada thread. Todas as threads executam essa mesma função.
void* comportamento(void* id) {
    long threadId = (long) id;

    // Imprime uma mensagem de boas-vindas.
    printf("Olá, sou a thread id %02ld\n", threadId);

    // Pausa a execução da thread por 1 segundo para simular trabalho.
    sleep(1);

    printf("Thread %02ld finalizada\n", (threadId));

    // A função pthread_exit() encerra a thread de forma explícita. O NULL indica que a thread não está retornando nada.
    pthread_exit(NULL); 
}

int main() {
    // Declara um array de IDs de thread. Cada elemento 'pthread_t'
    pthread_t threads[NUM_THREADS];

    // A cada iteração, uma nova thread é criada.
    for (long id = 0; id < NUM_THREADS; id++) {
        // pthread_create() é a função para criar uma nova thread.
        // Seus argumentos são:
        // 1. &threads[id]: O endereço da nova thread (endereço de um posição do vetor).
        // 2. NULL: Atributos da thread. NULL indica o uso das configurações padrão.
        // 3. comportamento: A função que a nova thread irá executar.
        // 4. (void*) id: O argumento que será passado para a thread.

        if (pthread_create(&threads[id], NULL, comportamento, (void*) id) != 0) {
            perror("Erro ao criar a thread");
            exit(1);
        }
    }

    printf("Todas as threads foram criadas.\n");

    // Finaliza o programa depois que todas as threads (incluindo o main) tiverem finalizado.
    pthread_exit(NULL); 
}

Writing threads.c


<h3>pthread_join()</h3>

<p>O <a href="https://man7.org/linux/man-pages/man3/pthread_join.3.html">pthread_join()</a> é uma função da biblioteca pthreads em C que serve para sincronizar a thread principal (ou outra thread qualquer) com uma thread joinable. Bloqueia a thread que chamou o pthread_join() até que a outra termine. Portanto, o programa fica esperando.</p>

<h4 style="color: #2d3436;"><strong>Código 2</strong>: Utilizando a função pthread_join().</h4>

In [ ]:
%%file join.c

// Comando de compilação: gcc -Wall join.c -o out -lpthread
// Comando de execução: ./out

#include <pthread.h> // Biblioteca para trabalhar com threads POSIX. Essencial para criar, gerenciar e sincronizar threads.
#include <stdio.h>   // Para funções de entrada e saída, como printf().
#include <stdlib.h>  // Para funções de utilidade geral, como exit().
#include <unistd.h>  // Para a função sleep() que pausa a execução.

#define NUM_THREADS 12

void* comportamento(void* id) {
    long threadId = (long) id;
    printf("Olá, sou a thread id %02ld\n", threadId);
    sleep(1);
    printf("Thread %02ld finalizada\n", (threadId));
    pthread_exit(NULL); 
}

int main() {
    pthread_t threads[NUM_THREADS];
    
    // Atributos da thread, usado para configurar o comportamento das threads.
    pthread_attr_t attr;
    pthread_attr_init(&attr);
    
    // Define os atributos da thread como joinable.
    pthread_attr_setdetachstate(&attr, PTHREAD_CREATE_JOINABLE);

    for (long id = 0; id < NUM_THREADS; id++) {
        if (pthread_create(&threads[id], &attr, comportamento, (void*) id) != 0) {
            perror("Erro ao criar a thread");
            exit(1);
        }
    }

    // Sincronização das tarefas executadas pelas threads
    for (int i = 0; i < NUM_THREADS; i++) {
        // Espera que cada thread termine antes de continuar.
        if (pthread_join(threads[i], NULL) != 0) {
            perror("Erro ao esperar pela thread");
            exit(1);
        }
    }

    printf("Todas as threads foram criadas.\n");
    
    // Destrói os atributos da thread após o uso.
    pthread_attr_destroy(&attr);
    pthread_exit(NULL); 
}

Overwriting threads.c


<h3>Referencias</h3>

<ul>
    <li>MAZIERO, Carlos A. Sistemas operacionais: conceitos e mecanismos. Livro aberto, 2014. <a href="https://wiki.inf.ufpr.br/maziero/lib/exe/fetch.php?media=socm:socm-livro.pdf">Livro</a></li>
</ul>